# Merging multiple DataFrames  

Merging tables is an important step in combining data from different sources to answer complex questions. In earlier lessons, we looked at merging two tables with a one-to-many relationship.

Sometimes, however, the analysis requires more than two tables to be merged together. This helps bring in additional information and allows for deeper insights.

When working with multiple tables, it is often necessary to carefully choose the columns on which to merge. Using only one column may lead to unwanted matches, so combining multiple columns can help ensure that the data aligns correctly.

The merge method in pandas allows us to specify multiple columns at once by passing a list of column names. In such cases, rows will only be matched if all specified columns have the same values in both tables.

Beyond merging two tables, we can continue to add more. After performing the first merge, we can merge the result with another table, repeating the process as many times as needed. This way, information from different sources can be brought together into a single DataFrame for analysis.

Finally, once all tables are merged, the combined dataset can be used to perform calculations, summaries, or visualizations. While three tables are often enough in many cases, pandas supports merging even more if the analysis requires it.

## Prepare Data

In [5]:
# Import pandas library
import pandas as pd

# Read the file
ridership = pd.read_pickle("datasets/cta_ridership.p")
cal = pd.read_pickle("datasets/cta_calendar.p")
stations = pd.read_pickle("datasets/stations.p")
licenses = pd.read_pickle("datasets/licenses.p")
wards = pd.read_pickle("datasets/ward.p")
zip_demo = pd.read_pickle("datasets/zip_demo.p")
census = pd.read_pickle("datasets/census.p")
land_use = pd.read_pickle("datasets/land_use.p")

## Exercise: Total riders in a month

The task is to calculate the total number of passengers who traveled through the Wilson station during weekdays in the month of July. This information is available across three different tables, which need to be merged in order to answer the question.

The three DataFrames provided are **cal**, **ridership**, and **stations**. The **cal** table is connected to **ridership** through the columns `year`, `month`, and `day`. Meanwhile, the **ridership** table links to **stations** through the column `station_id`.

### Instructions

1. Merge the **ridership** and **cal** tables first, using the common columns `year`, `month`, and `day`.
2. Merge the resulting DataFrame with the **stations** table on `station_id`.
3. Apply conditions to filter the data for the month of July, weekdays, and the Wilson station.
4. Finally, compute the total number of rides that meet these conditions.

In [2]:
# Step 1: Merge ridership with cal on year, month, and day
ridership_with_cal = ridership.merge(cal, on=['year', 'month', 'day'])

# Step 2: Merge the result with stations on station_id
full_data = ridership_with_cal.merge(stations, on='station_id')

# Step 3: Define filtering conditions
conditions = (
    (full_data['month'] == 7) &
    (full_data['day_type'] == 'Weekday') &
    (full_data['station_name'] == 'Wilson')
)

# Step 4: Apply the filter and calculate total rides
total_rides = full_data.loc[conditions, 'rides'].sum()

# Display the result
print(total_rides)

140005


## Exercise: Three table merge

You’ll now practice combining three datasets. Along with business license data and ward information, you also have demographic data that shows median income by zip code. Your task is to combine all three datasets and then examine how income varies across aldermen.

### Instructions

* Merge `licenses` with `zip_demo` on the `zip` column.
* Merge that result with `wards` on the `ward` column.
* Save the merged DataFrame as `combined_df`.
* Group `combined_df` by `alderman` and calculate the median income.

In [4]:
# Step 1: Merge licenses with zip_demo on 'zip'
licenses_zip = licenses.merge(zip_demo, on="zip")

# Step 2: Merge the result with wards on 'ward'
combined_df = licenses_zip.merge(wards, on="ward")

# Step 3: Group by alderman and calculate median income
median_income = combined_df.groupby("alderman")["income"].median()

# Display the result
print(median_income)

alderman
Ameya Pawar                   66246.0
Anthony A. Beale              38206.0
Anthony V. Napolitano         82226.0
Ariel E. Reyboras             41307.0
Brendan Reilly               110215.0
Brian Hopkins                 87143.0
Carlos Ramirez-Rosa           66246.0
Carrie M. Austin              38206.0
Chris Taliaferro              55566.0
Daniel "Danny" Solis          41226.0
David H. Moore                33304.0
Deborah Mell                  66246.0
Debra L. Silverstein          50554.0
Derrick G. Curtis             65770.0
Edward M. Burke               42335.0
Emma M. Mitts                 36283.0
George Cardenas               33959.0
Gilbert Villegas              41307.0
Gregory I. Mitchell           24941.0
Harry Osterman                45442.0
Howard B. Brookins, Jr.       33304.0
James Cappleman               79565.0
Jason C. Ervin                41226.0
Joe Moore                     39163.0
John S. Arena                 70122.0
Leslie A. Hairston            28024.0
Mar

## Exercise: One-to-many merge with multiple tables

**Finding the best ward for your goat business**
Suppose you want to launch a goat lawn-mowing service in Chicago. To choose a location, you need a ward with lots of open land, but not too many people or businesses nearby. You’ll combine data on vacant land, population, and existing business licenses to figure this out.

### Instructions

**Step 1 (35 XP)**
Merge the `land_use` and `census` tables on the `ward` column. Then merge this result with the `licenses` table, again on `ward`. Use suffixes `_cen` (for census) and `_lic` (for licenses). Store the merged DataFrame as `land_cen_lic`.

**Step 2 (35 XP)**
Group `land_cen_lic` by `ward`, `pop_2010`, and `vacant`. Count how many business accounts are in each group, and save the result to `pop_vac_lic`.

**Step 3 (30 XP)**
Sort `pop_vac_lic` by:

* `vacant` → descending
* `account` → ascending
* `pop_2010` → ascending

Save this sorted DataFrame as `sorted_pop_vac_lic`.

In [7]:
# Step 1: Merge land_use, census, and licenses on ward
land_cen_lic = (
    land_use.merge(census, on="ward")
            .merge(licenses, on="ward", suffixes=("_cen", "_lic"))
)

# Step 2: Group by ward, population, and vacant land; count business accounts
pop_vac_lic = (
    land_cen_lic.groupby(["ward", "pop_2010", "vacant"], as_index=False)
                .agg({"account": "count"})
)

# Step 3: Sort by vacant (desc), account (asc), and population (asc)
sorted_pop_vac_lic = pop_vac_lic.sort_values(
    ["vacant", "account", "pop_2010"],
    ascending=[False, True, True]
)

# Display the top results
print(sorted_pop_vac_lic.head())

   ward  pop_2010  vacant  account
47    7     51581      19       80
12   20     52372      15      123
1    10     51535      14      130
16   24     54909      13       98
7    16     51954      13      156
